<a href="https://colab.research.google.com/github/agmCorp/colab/blob/main/solucion1_matching_catalogos_bse_autodata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sistema de Matching Inteligente: BSE vs Autodata

**Objetivo:** Identificar automáticamente qué modelos de Autodata ya existen en el catálogo BSE


## 1️⃣ Instalación de dependencias

In [ ]:
# Instalar librerías necesarias
!pip install -q sentence-transformers pandas numpy scikit-learn unidecode tqdm

## 2️⃣ Importar librerías

In [ ]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, util
from unidecode import unidecode
import re
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings('ignore')

print("Librerías importadas correctamente")

## 3️⃣ Cargar los archivos CSV

**Importante:** Sube los archivos en la sección de archivos de Colab:
- `CART_MATRICERO_BSE_202602031858.csv`
- `CART_PAD_AUTODATA_202602031856.csv`

In [ ]:
# Cargar los datos
df_bse = pd.read_csv('/content/sample_data/RAW_CART_MATRICERO_BSE_202602131851.csv')
df_autodata = pd.read_csv('/content/sample_data/RAW_CART_PAD_AUTODATA_202602131851.csv')

print(f"Registros BSE: {len(df_bse):,}")
print(f"Registros Autodata: {len(df_autodata):,}")
print("\nPrimeras filas de BSE:")
display(df_bse.head(3))
print("\nPrimeras filas de Autodata:")
display(df_autodata.head(3))

## 4️⃣ Funciones de Preprocesamiento

In [ ]:
# NUEVA

def estandarizar_terminos_tecnicos(texto):
    """
    Estandariza abreviaturas técnicas de vehículos para mejorar el matching.
    Cubre transmisiones, puertas, tipos de motor y asistencias .
    """
    if not texto: return ""

    # Diccionario de equivalencias (Mapeo de abreviatura a término estándar)
    # Se utiliza \b en Regex para asegurar que coincida con la palabra completa
    mapeos = {
        # Asistencias y Seguridad [cite: 32]
        r'\bp\.assi\b': 'park assist',
        r'\bp\.assist\b': 'park assist',

        # Exterior [cite: 33, 42]
        r'\bllan\b': 'llantas',
        r'\blgo\b': 'largo',

        # Motores y Combustible [cite: 34, 35]
        r'\btd\b': 'turbo diesel',
        r'\bt\.diesel\b': 'turbo diesel',
        r'\bturbo d\b': 'turbo diesel',
        r'\btdi\b': 'turbo diesel intercooler',
        r'\btd intercooler\b': 'turbo diesel intercooler',
        r'\bt\.diesel intercooler\b': 'turbo diesel intercooler',

        # Puertas (Normalización a formato "X puertas") [cite: 36-41]
        r'\b2p\b|\b2 ptas\b': '2 puertas',
        r'\b3p\b|\b3 ptas\b': '3 puertas',
        r'\b4p\b|\b4 ptas\b': '4 puertas',
        r'\b5p\b|\b5 ptas\b': '5 puertas',
        r'\b3-5p\b|\b3-5 ptas\b': '3-5 puertas',
        r'\b4-5p\b|\b4-5 ptas\b': '4-5 puertas',

        # Transmisión [cite: 29, 30, 43]
        r'\baut\.\b|\baut\b': 'automatico',
        r'\btiptronic\b|\bs-tronic\b': 'automatico',
    }

    texto_procesado = texto.lower()
    for patron, reemplazo in mapeos.items():
        texto_procesado = re.sub(patron, reemplazo, texto_procesado)

    return texto_procesado

def normalizar_texto(texto):
    if pd.isna(texto): return ""
    texto = str(texto).lower()
    texto = unidecode(texto)
    # Aplicamos estandarización técnica antes de limpiar símbolos
    texto = estandarizar_terminos_tecnicos(texto)
    texto = re.sub(r'[^a-z0-9\s]', ' ', texto)
    return re.sub(r'\s+', ' ', texto).strip()

def corregir_anio_bse(anio):
    """Corrige años > 3000 restándoles 1000 (ej: 3020 -> 2020)"""
    if anio is None: return None
    return anio - 1000 if anio >= 3000 else anio

def extraer_anio_inicio_fin(rango_anios):
    if pd.isna(rango_anios): return None, None
    rango_str = str(rango_anios).strip()

    # Intenta encontrar dos años (rango)
    match = re.findall(r'(\d{4})', rango_str)
    if len(match) >= 2:
        return corregir_anio_bse(int(match[0])), corregir_anio_bse(int(match[1]))
    elif len(match) == 1:
        anio = corregir_anio_bse(int(match[0]))
        return anio, anio
    return None, None

# Función para limpiar marcas (maneja <BRA>, <COL>, etc.)
def normalizar_marca(texto):
    if pd.isna(texto): return ""
    texto = str(texto).upper()
    # Elimina cualquier contenido entre picos <...>
    texto = re.sub(r'<.*?>', '', texto)
    return texto.strip()

print("Funciones de preprocesamiento definidas")

## 5️⃣ Preprocesamiento de datos

In [ ]:
# Preprocesar BSE
df_bse['marca_norm'] = df_bse['MARCA'].apply(normalizar_marca)
df_bse['modelo_norm'] = df_bse['MODELO'].apply(normalizar_texto)
df_bse[['anio_inicio', 'anio_fin']] = df_bse['RANGO_ANIOS'].apply(
    lambda x: pd.Series(extraer_anio_inicio_fin(x))
)

print("Iniciando preprocesamiento de catálogos...")

# --- PROCESAR BSE ---
df_bse['marca_norm'] = df_bse['MARCA'].apply(normalizar_marca)
df_bse['modelo_norm'] = df_bse['MODELO_ORIGINAL'].apply(normalizar_texto)
df_bse[['anio_inicio', 'anio_fin']] = df_bse['RANGO_ANIOS'].apply(
    lambda x: pd.Series(extraer_anio_inicio_fin(x))
)
# Filtrar registros sin años procesables
df_bse = df_bse[df_bse['anio_inicio'].notna()].copy()

# --- PROCESAR AUTODATA ---
df_autodata['marca_norm'] = df_autodata['MARCA_A'].apply(normalizar_marca)
df_autodata['modelo_norm'] = df_autodata['MODELO_A_ORIGINAL'].apply(normalizar_texto)
df_autodata['anio'] = df_autodata['ANIO_A'].astype(int)

print(f"Preprocesamiento completado.")
print(f"BSE limpio: {len(df_bse):,} registros | Autodata: {len(df_autodata):,} registros.")


print("\nEjemplo de preprocesamiento:")
print("\nBSE:")
display(df_bse[['MARCA', 'marca_norm', 'MODELO_ORIGINAL', 'modelo_norm', 'RANGO_ANIOS', 'anio_inicio', 'anio_fin']].head(3))
print("\nAutodata:")
display(df_autodata[['MARCA_A', 'marca_norm', 'MODELO_A_ORIGINAL', 'modelo_norm', 'ANIO_A', 'anio']].head(3))

## 6️⃣ Cargar modelos

In [ ]:
from sentence_transformers import CrossEncoder, SentenceTransformer
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Usando dispositivo: {device}")

# 1. Cargamos Bi-Encoder para Fase 1 (Búsqueda rápida)
model_bi = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2', device=device)

# 2. Cargamos Cross-Encoder para Fase 2 (Reranker preciso)
reranker = CrossEncoder('BAAI/bge-reranker-v2-m3', device=device)

# 3. Pre-calculamos los embeddings de todo el catálogo BSE
print("Generando embeddings para el catálogo BSE (esto toma ~1-2 min)...")
bse_textos_para_embedding = (df_bse['marca_norm'] + " " + df_bse['modelo_norm']).tolist()
bse_embeddings = model_bi.encode(bse_textos_para_embedding, convert_to_tensor=True, show_progress_bar=True)

print("Modelos y embeddings listos.")


## 7️⃣ Matches

In [ ]:
# NUEVA
def encontrar_matches(row_autodata, df_bse, bse_embeddings, model, top_k=3, top_candidates=50):
    marca_auto = row_autodata['marca_norm']
    anio_auto = row_autodata['anio']
    modelo_auto = row_autodata['modelo_norm']
    texto_query = f"{marca_auto} {modelo_auto} {anio_auto if anio_auto != 9999 else ''}"

    # FASE 1: FILTRADO POR MARCA
    mask_candidatos = df_bse['marca_norm'].str.contains(marca_auto) | (df_bse['marca_norm'] == marca_auto)
    if mask_candidatos.sum() == 0: return []

    df_temp = df_bse[mask_candidatos].copy()
    indices_candidatos = df_temp.index.tolist()

    # Búsqueda vectorial rápida
    embeddings_subset = bse_embeddings[indices_candidatos]
    embedding_query = model.encode(texto_query, convert_to_tensor=False)
    similitudes = util.cos_sim(embedding_query, embeddings_subset)[0].numpy()

    # Tomamos un margen mayor para filtrar por año después
    top_n_raw = np.argsort(similitudes)[::-1][:top_candidates * 2]

    # FASE 1.5: FILTRADO POR COHERENCIA DE AÑO (Si el año no es 9999)
    valid_indices = []
    for i in top_n_raw:
        idx_bse = indices_candidatos[i]
        if anio_auto == 9999: # No sabemos el año, confiamos en texto
            valid_indices.append(idx_bse)
        else:
            ini, fin = df_bse.loc[idx_bse, 'anio_inicio'], df_bse.loc[idx_bse, 'anio_fin']
            # Permitimos un margen de error de 1 año en el catálogo
            if ini <= anio_auto <= fin:
                valid_indices.append(idx_bse)

    # Si el filtro de año fue muy agresivo, rescatamos los mejores por score
    if not valid_indices:
        valid_indices = [indices_candidatos[i] for i in top_n_raw[:top_candidates]]
    else:
        valid_indices = valid_indices[:top_candidates]

    # FASE 2: RERANKING
    pairs = [[texto_query, f"{df_bse.loc[idx, 'MARCA']} {df_bse.loc[idx, 'MODELO']} {df_bse.loc[idx, 'RANGO_ANIOS']}"]
             for idx in valid_indices]

    rerank_scores = reranker.predict(pairs)
    top_rerank_indices = np.argsort(rerank_scores)[::-1][:top_k]

    return [{
        'ID_AUTODATA_BSE': df_bse.loc[valid_indices[i], 'ID_AUTODATA_BSE'],
        'score': 1 / (1 + np.exp(-rerank_scores[i])), # Sigmoide
        'marca_bse': df_bse.loc[valid_indices[i], 'MARCA'],
        'modelo_bse': df_bse.loc[valid_indices[i], 'MODELO'],
        'rango_anios_bse': df_bse.loc[valid_indices[i], 'RANGO_ANIOS']
    } for i in top_rerank_indices]

## 8️⃣ Ejecutar matching para todo el catálogo Autodata

**NOTA:** Este proceso puede tomar varios minutos dependiendo del tamaño del catálogo. Usar GPU.

In [ ]:
TOP_K = 3
resultados_finales = []

print(f"Iniciando matching de {len(df_autodata)} registros...")

# Usamos tqdm para ver el progreso
for idx, row in tqdm(df_autodata.iterrows(), total=len(df_autodata)):
    # Ejecutar el pipeline de dos fases
    matches = encontrar_matches(row, df_bse, bse_embeddings, model_bi, top_k=TOP_K)

    # Estructura base del resultado (Datos de Autodata) [cite: 50, 52, 54, 56]
    res = {
        'ID_AUTADATA': row['ID_AUTADATA'],
        'MARCA_A': row['MARCA_A'],
        'MODELO_A_ORIGINAL': row['MODELO_A_ORIGINAL'],
        'ANIO_A': row['ANIO_A'],
    }

    if not matches:
        res.update({'SCORE_1': 0, 'ID_AUTODATA_BSE_1': 'NO_MATCH', 'DECISION': 'REVISAR'})
    else:
        # Llenar con los Top K matches encontrados [cite: 49, 51, 53, 55, 57, 58, 59]
        for i, m in enumerate(matches):
            suffix = f"_{i+1}"
            res[f'SCORE{suffix}'] = round(m['score'], 4)
            res[f'ID_AUTODATA_BSE{suffix}'] = m['ID_AUTODATA_BSE']
            res[f'MARCA_BSE{suffix}'] = m['marca_bse']
            res[f'MODELO_BSE{suffix}'] = m['modelo_bse']
            res[f'RANGO_ANIOS_BSE{suffix}'] = m['rango_anios_bse']
            # Solo para el primer match incluimos campos extra de trazabilidad
            if i == 0:
                # Buscamos los datos originales del BSE para el match principal
                original_bse = df_bse[df_bse['ID_AUTODATA_BSE'] == m['ID_AUTODATA_BSE']].iloc[0]
                res['COMB_BSE'] = original_bse.get('COMB', '')
                res['TIPO_BSE'] = original_bse.get('TIPO', '')

    resultados_finales.append(res)

# Crear DataFrame y exportar
df_resultados = pd.DataFrame(resultados_finales)
df_resultados.to_csv('resultados_matching_final.csv', index=False, encoding='utf-8-sig')

print("\n¡Proceso finalizado!")
display(df_resultados.head())

## 9️⃣ Exportar resultados a CSV

In [ ]:
# Exportar resultados completos
archivo_salida = 'resultados_matching_bse_autodata.csv'
df_resultados.to_csv(archivo_salida, index=False, encoding='utf-8-sig')

print(f"Resultados exportados a: {archivo_salida}")
print(f"Total de registros: {len(df_resultados):,}")

print("\n" + "="*70)
print("PROCESO COMPLETADO EXITOSAMENTE")
print("="*70)